In [23]:
%matplotlib inline
from __future__ import print_function
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import flopy
import pyemu

In [24]:
#model_ws = os.path.join("..","model")
#nam_file = "10par_xsec.nam"
model_ws = os.path.join("..","FreybergModelForClass","Freyberg_Truth")
nam_file = "freyberg.truth.nam"
ml = flopy.modflow.Modflow.load(nam_file,model_ws=model_ws,check=False,verbose=True)


Creating new model with name: freyberg.truth
--------------------------------------------------

Parsing the namefile --> ../FreybergModelForClass/Freyberg_Truth/freyberg.truth.nam
Setting filehandles:

--------------------------------------------------
External unit dictionary:
{32: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.pcg, filetype:PCG, 51: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.hds, filetype:DATA(BINARY), 2: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.list, filetype:LIST, 36: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.hyd, filetype:HYD, 11: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.dis, filetype:DIS, 13: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.bas, filetype:BAS6, 14: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.oc, filetype:OC, 18: filename:../FreybergModelForClass/Freyberg_Truth/freyberg.truth.riv, filetype:RIV, 19: filename:../FreybergModelForClass/Freyberg_Truth/fr

In [25]:
variance_fraction = {"flux":0.1,"stage":0.01,"cond":0.2,"rech":0.025}

In [26]:
# wells
wdata = ml.wel.stress_period_data.array["flux"]
noise = np.random.standard_normal(wdata.shape)
# scale by the variance of each pumping well
noise *= wdata * variance_fraction["flux"]
wdata += noise
# create a new MfList with noisy data
wdata = flopy.utils.MfList.masked4D_arrays_to_stress_period_data(flopy.modflow.ModflowWel.
                                                                 get_default_dtype(),{"flux":wdata})
# use noisy data to make new well package
wel = flopy.modflow.ModflowWel(ml,stress_period_data=wdata)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfwel.ModflowWel'> <class 'flopy.modflow.mfwel.ModflowWel'>
replacing existing Package...


In [27]:
# riv - add noise to rcond and stage
array_dict = ml.riv.stress_period_data.array
for prop in ["cond","stage"]:    
    noise = np.random.standard_normal(array_dict[prop].shape)
    #scale by the variance of each pumping well
    noise *= array_dict[prop] * variance_fraction[prop]
    array_dict[prop] += noise
array_dict = flopy.utils.MfList.masked4D_arrays_to_stress_period_data(flopy.modflow.ModflowRiv.
                                                                 get_default_dtype(),array_dict)
riv = flopy.modflow.ModflowRiv(ml,stress_period_data=array_dict)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfriv.ModflowRiv'> <class 'flopy.modflow.mfriv.ModflowRiv'>
replacing existing Package...


In [28]:
# recharge
rech_noise = np.random.standard_normal(2) 

rech_vals = {0:ml.rch.rech[0].array[0,0],1:ml.rch.rech[1].array[0,0]}

rech = {}
for sp in rech_vals.keys():
    n = rech_vals[sp]*rech_noise[sp]*variance_fraction["rech"]
    print(n)
    rech[sp] = rech_vals[sp] + n
print(rech_vals)
print(rech)
rch = flopy.modflow.ModflowRch(ml,rech=rech,nrchop=3)

-2.29009207461e-06
1.37383041151e-06
{0: 0.00013823999, 1: 9.0000001e-05}
{0: 0.0001359499011735221, 1: 9.1373831775899846e-05}
****Warning -- two packages of the same type:  <class 'flopy.modflow.mfrch.ModflowRch'> <class 'flopy.modflow.mfrch.ModflowRch'>
replacing existing Package...


In [29]:
ml.name = "freyberg"
ml.change_model_ws(os.path.join("..","FreybergModelForClass","Freyberg_Noise"),reset_external=True)
ml.write_input()


changing model workspace...
   ../FreybergModelForClass/Freyberg_Noise

Writing packages:
   Package:  DIS
   Package:  BAS6
   Package:  PCG
   Package:  HYD
   Package:  OC
   Package:  RIV
   Package:  RCH
   Package:  WEL
   Package:  LPF
 


In [30]:
ml.exe_name = "mf2005"
ml.run_model()

FloPy is using the following executable to run the model: /Users/jwhite/bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2016/05/11 14:17:42

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2016/05/11 14:17:42
 Elapsed run time:  0.025 Seconds

  Normal termination of simulation


(True, [])